# Notebook zum QR-Algorithmus

### Vorbereitungen

Wir benötigten in diesem Notebook die folgenden Module:

In [1]:
import numpy as np
import scipy.linalg as spla  # für Matrixzerlegungen und co
import numpy.random as rnd   # für alles, was mit Zufallszahlen zu tun hat

Die Prozeduren, die wir in diesem Notebook betrachten, liefern verschiedene Vektoren und Matrizen als Ergebnis. Um diese schöner darstellen zu können, eignen sich die folgenden beiden Prozeduren. Was diese Prozeduren genau tun müssen Sie sich nicht anschauen.

In [2]:
def printvector(v):
    if v.dtype == "int":
        print(''.join([' {:4}'.format(item) for item in v])+"\n")
    elif v.dtype == "complex128":
        print(''.join([' {:16.3f}'.format(item) for item in v])+"\n")
    else:
        print(''.join([' {:7.3f}'.format(item) for item in v])+"\n")

In [3]:
def printmatrix(A):
    if A.dtype == "int":
        print('\n'.join([''.join(['  {:4}'.format(item) for item in row]) for row in A])+"\n")
    elif A.dtype == "complex128":
        print('\n'.join([''.join(['  {:16.3f}'.format(item) for item in row]) for row in A])+"\n")   
    else:
        print('\n'.join([''.join(['  {:7.3f}'.format(item) for item in row]) for row in A])+"\n")       

### Problemstellung & Modellmatrizen
In diesem Notebook wollen wir Eigenwerte von Matrizen mit verschiedenen Varianten des QR-Algorithmus approximieren. Zunächst konstruieren wir uns dazu ein paar Modellmatrizen, von denen wir die Eigenwerte kennen und mit denen wir die Algorithmen testen können.

Zur Konstruktion der Matrizen starten wir zunächst mit einer Diagonalmatrix oder einer Blockdiagonalmatrix $D$, und wenden dann eine beliebig ausgewählte Ähnlichkeitstransformation an, d.h. berechnen $A=S^{-1} D S$ für eine invertierbare Matrix $S$. Die Eigenwerte von $A$ entsprechen dann denen von $D$. Der Matrix $A$ selbst sieht man aber die Eigenwerte nicht direkt an.

Konkret verwenden wir
$$
D_1 = \begin{pmatrix} 2 \\ & 1 \\ && 5 \\ &&& -4 \\ &&&& \frac12 \end{pmatrix}, \qquad
D_2= \begin{pmatrix} 2+2\mathrm{i} \\ & 1 \\ && 5 \\ &&& -4+1\mathrm{i} \\ &&&& \frac12-3\mathrm{i} \end{pmatrix}, \qquad
D_3 = \begin{pmatrix} 2 \\ & 1 \\ && 5 \\ &&& 5 \\ &&&& \frac12 \end{pmatrix}, \qquad
D_4 = \begin{pmatrix} 2 \\ & 1 \\ && 5 \\ &&& -5 \\ &&&& \frac12 \end{pmatrix}, \qquad
$$
sowie
$$
D_5 = \begin{pmatrix} 2+2\mathrm{i} \\ & 1 \\ && 1 & -1 \\ && 1 & 1 \\ &&&& \frac12-3\mathrm{i} \end{pmatrix} 
\qquad \text{und} \qquad
D_6 = \begin{pmatrix} 2 \\ & 1 \\ && 1 & -1 \\ && 1 & 1 \\ &&&& \frac12 \end{pmatrix},
$$
und definieren dann $A_i = S^{-1} D_i S$ für $i=1,\ldots,6$ mit der invertierbaren Matrix
$$ 
S = \begin{pmatrix} 
     2 & -1 &  1 &  0 & -1 \\
    -1 &  1 &  2 &  2 &  2 \\
    -1 &  0 &  2 & -1 &  1 \\
    -1 &  2 &  2 &  2 &  1 \\
     2 & -1 &  2 &  0 & -1 
\end{pmatrix}.
$$ 

In [112]:
# (Block-)Diagonalmatrizen
A1 = np.diag(np.array([2, 1, 5, -4, 1/2]))
A2 = np.diag(np.array([2+2j, 1, 5, -4+1j, 1/2-3j]))
A3 = np.diag(np.array([2, 1, 5,  5, 1/2]))
A4 = np.diag(np.array([2, 1, 5, -5, 1/2]))
A5 = np.array([[2+2j,0,0,0,0],[0,1,0,0,0],[0,0,1,-1,0],[0,0,1,1,0],[0,0,0,0,1/2-3j]])
A6 = np.array([[2,0,0,0,0],[0,1,0,0,0],[0,0,1,-1,0],[0,0,1,1,0],[0,0,0,0,1/2]])

# Ähnlichkeitstransformation
S = np.array([
    [2, -1, 1, 0, -1],
    [-1, 1, 2, 2, 2],
    [-1, 0, 2, -1, 1],
    [-1, 2, 2, 2, 1],
    [2, -1, 2, 0, -1]
])
S_inv = spla.inv(S)

A1 = S_inv @ A1 @ S
A2 = S_inv @ A2 @ S
A3 = S_inv @ A3 @ S
A4 = S_inv @ A4 @ S
A5 = S_inv @ A5 @ S
A6 = S_inv @ A6 @ S

Für die Matrix $A_1$ gilt dann zum Beispiel:

In [32]:
print('A_1 =')
printmatrix(A1)

A_1 =
   16.000  -15.125   10.250  -11.500   -7.375
   15.000  -17.875    3.750  -16.500   -8.625
   -3.000    1.500   -1.000    0.000    1.500
   -7.000    9.250   -8.500   10.000    2.750
   10.000   -8.875   13.750   -6.500   -2.625



**(a) Nennen Sie kurz die "Herausforderungen" im Bezug auf die Eigenwerte von $D_3, \ldots, D_6$.**

D3: doppelter Eigenwert
D4: verschiedene Eigenwerte mit gleichem Betrag
D5, D6: 2x2 Block hat keine Eigenvektoren zwei verschiedene komplexe Eigenwerte mit gleichem Betrag

## 1.) Naiver QR-Algorithmus

**(b) Implementieren Sie eine Prozedur `qr_alg_naiv`, die den QR-Algorithmus in der naiven Version auf eine Matrix anwendet. Die Anzahl der Iterationen `kMax` soll dabei als Eingabeparameter übergeben werden.**

Berechnen Sie die QR-Zerlegungen mithilfe der in `scipy` enthalten Prozedur (also mit `spla.qr(...)`).

In [41]:
def qr_alg_naive(M, kMax=100):
    A = M.copy()
    for k in range(kMax):
        Q, R = spla.qr(A)
        A = R @ Q
    return A

**(c) Wenden Sie die Prozedur zunächst auf die Matrizen $A_1$ und $A_2$ an. Wie viele Iterationen `kMax` brauchen Sie für gute Ergebnisse?**

In [47]:
printmatrix(qr_alg_naive(A1, 35))

    5.003   16.733   -9.269  -18.006   38.105
   -0.001   -4.003   -1.398   13.071   -2.919
    0.000   -0.000    2.000   -1.176    7.512
    0.000    0.000    0.000    1.000   -0.042
    0.000    0.000   -0.000   -0.000    0.500



In [ ]:
printmatrix(qr_alg_naive(A2, 125))

      5.000+0.000j   -13.061-10.625j    12.872+17.095j  -53.831+101.745j   -21.164-14.967j
      0.000-0.000j     -4.000+1.000j     -0.367+1.306j      2.087+5.592j    -8.209+10.535j
      0.000+0.000j      0.000-0.000j      0.500-2.999j     6.871-24.770j      2.711+3.907j
      0.000+0.000j     -0.000-0.000j     -0.000-0.000j      2.000+1.999j     -0.675-0.225j
     -0.000-0.000j      0.000+0.000j      0.000+0.000j     -0.000-0.000j      1.000+0.000j



**(d) Wenden Sie die Prozedur nun jeweils mit `kMax = 50` auf die Matrizen $A_3,...,A_6$ an. Beobachten und erklären Sie die Ergebnisse.**

In [ ]:
printmatrix(qr_alg_naive(A3, 50))
# schnelle Konvergenz, da sich die Eigenwerte bis auf mehrfaches Auftreten im Betrag recht deutlich unterscheiden

    5.000    0.000  -13.017   26.367  -33.573
    0.000    5.000   -2.136   -4.420   -6.529
    0.000    0.000    2.000   -1.176   -7.512
   -0.000    0.000    0.000    1.000    0.042
   -0.000   -0.000    0.000    0.000    0.500



In [ ]:
printmatrix(qr_alg_naive(A4, 50))
# langsame Konvergenz, da zwei verschieden Eigenwerte den gleichen Betrag haben


    0.654   19.832   -8.281  -25.582  -38.321
    1.239   -0.654   -3.486   10.392   -5.175
    0.000    0.000    2.000   -1.176   -7.512
    0.000   -0.000    0.000    1.000    0.042
   -0.000    0.000    0.000    0.000    0.500



In [60]:
printmatrix(qr_alg_naive(A5, 50))
# siehe A4


      0.604-2.872j  -42.473-104.317j    14.336+33.575j      5.945-9.714j   -13.733+19.840j
     -0.004-0.006j      1.851+1.724j     -0.306-0.444j     -0.372+0.125j      0.714-0.399j
      0.001+0.002j      0.072-0.495j      1.612+0.147j     -1.003-0.356j      0.565+0.299j
     -0.000-0.000j      0.323-0.000j      1.077-0.342j      0.432-0.000j     -1.456+0.000j
      0.000+0.000j     -0.000+0.000j     -0.000+0.000j      0.000+0.000j      1.000-0.000j



In [62]:
printmatrix(qr_alg_naive(A6, 50))
# konvergiert nicht, da nicht alle Eigenwerte reell, aber nur mit reellen Zahlen gerechnet wird

    2.000    3.000   -3.397  -10.157  -35.332
   -0.000    1.549    1.117    0.816    0.354
    0.000   -1.165    0.451    1.478   -0.075
   -0.000    0.000    0.000    1.000    0.042
    0.000   -0.000   -0.000    0.000    0.500



## 2.) Transformation auf Hessenberg-Form

Als ersten Schritt hin zu einer effizienteren Implementierung wollen wir die Matrizen durch eine Ähnlichkeitstransformation auf Hessenberg-Form bringen.

**(e) Schreiben Sie eine Prozedur `hess`, die eine Matrix durch eine unitäre Ähnlichkeitstransformation auf Hessenberg-Form bringt (siehe Beweis von Satz 6.22).**

Hinweise:
* Berücksichtigen Sie, wie die richtige Householder-Transformation aussieht, die einen Vektor mit **komplexen** Einträgen auf ein Vielfaches des ersten Einheitsvektors spiegelt (siehe Abschnitt 6.8.1 im Skript).
* Die verwendeten Householder-Transformationen brauchen Sie nicht speichern, denn wir sind nur an den Eigenwerten von Matrizen interessiert, und die Hessenberg-Matrix, die Ihre Prozedur am Ende liefert, ist ja ähnlich zur Ausgangsmatrix.
* Bei Vektoren (genauer: eindimensionale `ndarray`) unterscheidet Numpy nicht zwischen Zeilen- und Spaltenvektoren, sondern interpretiert sie (zum Beispiel bei der Matrixmultilplikation) so wie es Sinn macht.
* Die komplex konjugierte Variante eines Vektors/arrays $v$ erhalten Sie über `v.conj()`.
* Das dyadische Produkt $vw^\top$ zweier Vektoren $v,w\in\mathbb{C}^n$ erhalten Sie über `np.outer(v,w)`, und dementsprechend das Produkt $vw^H$ über `np.outer(v,w.conj())`.

In [125]:
def hess(A):
    n = A.shape[0]
    for i in range(n-2):
        x = A[i+1:, i]
        sgn = np.sign(x[0]) if x[0] != 0 else 1
        alpha = -sgn * spla.norm(x)
        e1 = np.zeros(n-1-i)
        e1[0] = 1
        u = x - alpha * e1
        u /= np.linalg.norm(u)
        Q_squiggle = np.eye(n-1-i) - 2 * np.outer(u, u.conj())
        Q = spla.block_diag(np.eye(i+1), Q_squiggle)
        A = Q @ A @ Q
    return A


**(f) Wenden Sie Ihre Prozedur zum Test auf die Matrizen $A_1$ und $A_2$ an. Übergeben Sie dabei nur eine Kopie der Matrizen an die Prozedur `hess`, damit die Ausgangsmatrizen unverändert bleiben. Überprüfen Sie: Haben die Ergebnismatrizen Hessenberg-Struktur? Haben Sie weiterhin die selben Eigenwerte wie die Ausgangsmatrizen (über `spla.eigvals` überprüfbar)?**

In [ ]:
M = hess(A1.copy())
printmatrix(M)

print(spla.eigvals(M))
print(spla.eigvals(A1))

   16.000   12.819  -11.423   14.699   -3.098
  -19.570  -16.389   16.033  -23.265    6.682
    0.000   -0.757    0.407   -3.797    6.616
    0.000   -0.000   -4.424    3.517    8.044
   -0.000   -0.000    0.000    0.039    0.965

[-4. +0.j  5. +0.j  2. +0.j  0.5+0.j  1. +0.j]


In [134]:
M = hess(A2.copy())
printmatrix(M)

print(spla.eigvals(M))
print(spla.eigvals(A2))

    16.000+46.750j     8.848+35.339j    -0.451-20.130j    22.602+25.078j    -3.733+10.226j
   -23.534-61.580j   -13.510-46.700j     2.218+27.477j   -35.927-31.542j     5.042-14.034j
      0.000+0.000j     -0.026+0.353j     -3.756+1.070j     -2.716-0.912j     11.195+0.174j
      0.000+0.000j     -0.000+0.000j     -3.012-0.302j      4.555-1.203j      5.529+0.976j
     -0.000+0.000j     -0.000+0.000j     -0.000-0.000j     -0.445+0.415j      1.212+0.082j

[-4. +1.00000000e+00j  0.5-3.00000000e+00j  5. +1.33305712e-14j
  2. +2.00000000e+00j  1. +7.03781454e-15j]
[-4. +1.00000000e+00j  0.5-3.00000000e+00j  5. -1.96636464e-14j
  2. +2.00000000e+00j  1. +6.75873537e-15j]


## 3.) QR-Algorithmus mit Shift für Hessenberg Matrizen

Bevor wir uns um den QR-Algorithmus mit Shift kümmern, bringen wir zunächst alle Modellmatrizen in Hessenberg-Form:

In [135]:
A1_hess = hess(A1.copy())
A2_hess = hess(A2.copy())
A3_hess = hess(A3.copy())
A4_hess = hess(A4.copy())
A5_hess = hess(A5.copy())
A6_hess = hess(A6.copy())

Nun wollen wir den QR-Algorithmus mit Shift für eine Hessenberg-Matrix $H$ implementieren. Dabei verwenden wird immer das letzte Element $\mu = h_{n,n}$ von $H$ als Shift. Wir iterieren so lange, bis Deflation auftritt, in dem Sinne, dass
$$
|h_{n,n-1}| \leq \texttt{eps} \left( |h_{n-1,n-1,}| + |h_{n,n}| \right)
$$
mit der Maschinengenauigkeit $\texttt{eps}$ gilt.

**(g) Implementieren Sie den eben beschriebenen QR-Algorithmus mit Shift. Sobald Deflation auftritt, soll die Prozedur beendet werden und dabei den isolierten Eigenwert sowie die Restmatrix ausgeben.**

Hinweise:
* Verwenden Sie wieder einen Parameter `kMax`, um die maximale Iterationszahl festzulegen, falls es nicht vorzeitig zu Deflation kommen sollte. In diesem Fall soll nur die letzte Iterierte ausgegeben werden.
* Die Maschinengenauigkeit (für 64Bit-Gleitkommazahlen) erhalten Sie über `np.finfo(np.float64).eps`.
* Stellen Sie sicher, dass Ihre Prozedur auch für $1\times1$-Matrizen ein sinnvolles Ergebnis liefert.

Testen Sie Ihre Prozedur mit den Matrizen `A1_hess` und `A2_hess`. Lassen Sie Python auch die Eigenwerte der Restmatrix berechnen und überprüfen Sie, ob die Ergebnisse Sinn ergeben.

In [ ]:
def qr_alg_shifts(A, kMax=100):
    H = A.copy()
    n = H.shape[0]
    if n == 1:
        return None, H
    for k in range(kMax):
        shift = H[n-1, n-1]
        H_shift = H - shift * np.eye(n)
        Q, R = spla.qr(H_shift)
        H = R @ Q + shift * np.eye(n)
        if spla.norm(H[n-1, n-2]) <= np.finfo(np.float64).eps * (spla.norm(H[n-2, n-2]) + spla.norm(H[n-1, n-1])):
            return H[n-1, n-1], H[:n-1, :n-1]
    return None, H


In [153]:
eig, R = qr_alg_shifts(A1_hess, 35)
print(eig)
printmatrix(R)
printvector(spla.eigvals(R))

print('---')

eig, R = qr_alg_shifts(A2_hess, 35)
print(eig)
printmatrix(R)
printvector(spla.eigvals(R))

1.0000000000000044
   -0.986   17.814    7.826   35.163
    1.025    2.042    4.191    9.966
   -0.000   -0.049    2.021   -7.550
    0.000    0.000    0.016    0.422

    -4.000+0.000j     5.000+0.000j     2.000+0.000j     0.500+0.000j

---
(1+3.969276400905076e-15j)
     -2.209+1.445j    -11.045-3.857j     9.396+16.481j    64.365-84.725j
      0.409+1.071j     -3.308-3.309j      0.184+3.165j     43.398-3.493j
      0.000-0.000j     -0.361+4.897j      7.799-0.278j   -12.630-27.281j
     -0.000-0.000j     -0.000-0.000j     -0.138-0.014j      1.218+2.143j

    -4.000+1.000j     0.500-3.000j     5.000-0.000j     2.000+2.000j



**(h) Erweitern Sie Ihre Prozedur aus Teil (g) folgendermaßen: Sobald Deflation auftritt, rufen Sie die Prozedur rekursiv auf, um den QR-Algorithmus mit der Restmatrix erneut zu starten. Speichern Sie dann den isolierten Eigenwert sowie die Eigenwerte der Restmatrix in einem Vektor, den Sie am Ende zurückgeben. Achten Sie hier besonders darauf, dass Ihre Prozedur für $1\times1$-Matrizen sinnvoll agiert.**

In [176]:
def qr_alg_shifts_recursive(A, kMax=100):
    H = A.copy()
    n = H.shape[0]
    eigvals = np.array([], dtype=H.dtype)
    if n == 1:
        return np.append(eigvals, H[0, 0])
    for k in range(kMax):
        shift = H[n-1, n-1]
        H_shift = H - shift * np.eye(n)
        Q, R = spla.qr(H_shift)
        H = R @ Q + shift * np.eye(n)
        if spla.norm(H[n-1, n-2]) <= np.finfo(np.float64).eps * (spla.norm(H[n-2, n-2]) + spla.norm(H[n-1, n-1])):
            eigvals = np.append(eigvals, H[n-1, n-1])
            eigvals = np.append(eigvals, qr_alg_shifts_recursive(H[:n-1, :n-1], kMax))
            return eigvals
    return eigvals


**(i) Sofern kein Abbruch wegen Erreichen der maximalen Iterationszahl erfolgt, sollte Ihre Prozedur aus Teil (h) einen Vektor mit allen Eigenwerten der eingegebenen Matrix $H$ berechnen. Überprüfen Sie dies zunächst anhand der Matrizen `A1_hess` und `A2_hess`. Wenn für diese Matrizen alles funktioniert, testen Sie auch die Matrizen `A3_hess`,...,`A6_hess`. Mit welchen Matrizen kommt die Prozedur (nicht) klar? Haben Sie eine Idee, warum? Beobachten Sie außerdem auch die Anzahl an Iterationen, die insgesamt durchgeführt wird, insbesondere im Vergleich zu den Aufgabenteilen (c) und (d).**

In [188]:
printvector(qr_alg_shifts_recursive(A1_hess, 4))
printvector(qr_alg_shifts_recursive(A2_hess, 5))


   1.000   0.500   2.000   5.000  -4.000

     1.000+0.000j     2.000+2.000j     5.000-0.000j     0.500-3.000j    -4.000+1.000j



In [ ]:
printvector(qr_alg_shifts_recursive(A3_hess, 5))
printvector(qr_alg_shifts_recursive(A4_hess, 4))
printvector(qr_alg_shifts_recursive(A5_hess, 7))
printvector(qr_alg_shifts_recursive(A6_hess.astype(np.complex128), 100))
# keine Ahnung warum es für A6 nicht konvergiert, aber für A5 schon


   5.000   5.000   1.000   0.500   2.000

   1.000   0.500   2.000   5.000  -5.000

     1.000-0.000j     1.000+1.000j     1.000-1.000j     2.000+2.000j     0.500-3.000j

     1.000+0.000j     0.500+0.000j     2.000+0.000j



Natürlich können wir unsere Prozedur jetzt auch auf andere Matrizen als die Modellmatrizen anwenden, und so die Eigenwerte (quasi) beliebiger Matrizen berechnen. Hier zum Beispiel für eine zufällige Matrix mit komplexen Einträgen: 

In [218]:
n = 20
A = rnd.rand(n,n) + 1j*rnd.rand(n,n)
A_hess = hess(A)

print('Eigenwerte laut Python:')
printvector(np.sort(spla.eigvals(A_hess)))

print('Eigenwerte selbst berechnet:')
printvector(np.sort(qr_alg_shifts_recursive(A_hess,50)))

Eigenwerte laut Python:
    -1.799-0.628j    -1.302+0.263j    -0.911+0.483j    -0.840+0.910j    -0.724-1.160j    -0.416-1.232j    -0.382+0.089j    -0.048+0.661j    -0.041-0.423j    -0.016+1.207j     0.044+0.335j     0.484-1.512j     0.520+1.118j     0.676-1.075j     0.768-0.036j     0.769-0.301j     1.481+0.848j     1.645+0.293j     1.893-1.178j     9.979+9.990j

Eigenwerte selbst berechnet:
    -1.799-0.628j    -1.302+0.263j    -0.911+0.483j    -0.840+0.910j    -0.724-1.160j    -0.416-1.232j    -0.382+0.089j    -0.048+0.661j    -0.041-0.423j    -0.016+1.207j     0.044+0.335j     0.484-1.512j     0.520+1.118j     0.676-1.075j     0.768-0.036j     0.769-0.301j     1.481+0.848j     1.645+0.293j     1.893-1.178j     9.979+9.990j

